### Imports

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize,PowerNorm
import matplotlib.cm as cm
from tqdm.auto import tqdm
import sys
import os 
import warnings

warnings.filterwarnings("ignore")


### Custom

In [ ]:
sys.path.append(os.path.abspath('./PINNS/'))
sys.path.append(os.path.abspath('./PDE/'))
sys.path.append(os.path.abspath('./Losses/'))
from Poisson2D import *
from Poisson2DSTLosses import P2DSTLoss
from PoissonCircLoss import CircLoss
from IPINN import IPINN
from VDLNet import VDLNet

In [ ]:
torch.cuda.empty_cache()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

### Plotting Functions

In [ ]:
def compare_contours(X,Y,exact,result,colormap = 'plasma',levels=10,nrange = (0,1)):
    """
    Produces a contour plot comparing exact and predicted results
    X,Y: Axes in form of meshgrids
    exact: Analytical Solution
    result: PINN solution
    It will also return the RMSE and L2 errors 
    """
    res = np.sqrt(np.sum(np.square(exact - result)))/np.sqrt(np.sum(np.square(exact)))
    rmse = np.sqrt(np.mean(np.square(exact- result)))
    err = np.abs(exact - result)
    
    fig, axes = plt.subplots(1, 3,figsize=(18, 4))
    cmap = cm.get_cmap(colormap)
    normalizer = Normalize(nrange[0],nrange[1])
    im = cm.ScalarMappable(norm = normalizer,cmap = colormap)
    fig.colorbar(im, ax=axes.ravel().tolist())

    axes[0].contourf(X, Y, result, levels=levels, cmap=colormap,norm = normalizer)
    axes[0].set_title('Prediction')

    axes[1].contourf(X, Y, exact , levels=levels, cmap=colormap,norm = normalizer)
    axes[1].set_title('Actual')
    
    axes[2].contourf(X, Y, err , levels=levels, cmap=colormap,norm=normalizer)
    axes[2].set_title('RMSE:{:.2e}, Rel. L2:{:.2e}'.format(rmse,res))
    
    plt.show()

    res2 = np.sum(err)
    print('L1: ',res2)

In [ ]:
def plot_results3d(X,Y,exact,result,colormap = 'cividis'):
    """
    Produces a surface plot comparing exact,predicted results and error between them
    X,Y: Axes in form of meshgrids
    exact: Analytical Solution
    result: PINN solution
    """
    fig,axes = plt.subplots(1,3,figsize=(18, 4),subplot_kw={'projection': '3d'})
    error = exact - result
    
    axes[0].plot_surface(X, Y, result, cmap=colormap)
    axes[0].set_xlabel('x')
    axes[0].set_ylabel('y')
    axes[0].set_zlabel('u')
    axes[0].set_title('Predicted')
    
    axes[1].plot_surface(X, Y, exact, cmap=colormap)
    axes[1].set_xlabel('x')
    axes[1].set_ylabel('y')
    axes[1].set_zlabel('u')
    axes[1].set_title('Analytical Solution')
    
    axes[2].plot_surface(X, Y, error, cmap=colormap)
    axes[2].set_xlabel('x')
    axes[2].set_ylabel('y')
    axes[2].set_zlabel('Error')
    axes[2].set_title('Error')
    
    plt.show()

## Two Dimensional Elliptical Interface Problems

### 1. Poisson with a straight interface
Domain: $Ω = [0, 1]×[0, 1]$, Interface: $Γ_{int} = {x : y = x}$ <br /> 
Sub-Domains: $Ω_1 = {x : y ≤ x} ⊂ Ω$ and $Ω_2 = {x : y > x} ⊂ Ω$ <br />

The problem can be stated as:
- $∇·(κ_m∇u_m) = 1$ in $Ω_m$
- $um = Λ_m^d$ on $∂Ω_m^d$
- $[[u]] = 0$ on $Γ_{int}$
- $[[κ∇u]] · \mathbf{n_2} = \frac{−y}{√2}$ on $Γ_{int}$

where $∂Ω_1^d = \{ \textbf{x} : x = 1 ∪ y = 0 \}$ and $∂Ω_2^d = \{ \textbf{x} : x = 0 ∪ y = 1 \}$ with dirchlet boundary conditions.<br /> 
Considering $κ_1 = 0.5$ and $κ_2 = 0.25$, the set of equations given have a closed-form analytical solution given as:
$$ u(x) = \left\{
            \begin{array}{ll}
                  x^2 + xy & \text{in }  Ω_1\\ 
                  x^2 + y^2 & \text{in } Ω_2\\
            \end{array}
            \right. $$
The known boundary data $Λ_1^d$ and $Λ_2^d$ at the Dirichlet boundaries are prescribed according to $u(x)$

In [ ]:
K = [0.5,0.25]
ls_st = P2DSTLoss(10000,100,100,K)
#Get the exact function
st_poiss = Poisson2DST()

In [ ]:
h = 0.001
# Generate a grid of points
x = torch.arange(0, 1+h, h)
y = torch.arange(0, 1+h, h)
X, Y = torch.meshgrid(x, y)
XY = torch.stack((X, Y), dim=-1).reshape(-1, 2).to(device)

def getRes(outputs):
    # Evaluate the models
    u1, u2 = outputs[0],outputs[1]
    # Initialize result array
    result = torch.zeros_like(u1).cpu().numpy().reshape(X.shape)
    
    # Assign values based on the domain
    u1 = u1.cpu().numpy().reshape(X.shape)
    u2 = u2.cpu().numpy().reshape(X.shape)
    mask = X > Y
    result[mask] = u1[mask]
    result[~mask] = u2[~mask]
    
    return result

#### IPINN

In [ ]:
#Get the activation function
def cond_func(x,condition):
    if (condition == '0'):
        return F.silu(x)
    elif (condition == '1'):
        return F.tanh(x)

In [ ]:
#define the model
ipinn = IPINN(dimension = 2,hidden_size = 10,depth = 2)
#Get the loss_function
ls_st.reset()
ipinn.setLoss(lambda x: ls_st.loss(x,'ipinn',cond_func))
#Train the model
ipinn.train(iterations = 30000)

In [ ]:
#get the activations
activations = [F.silu,F.tanh]
#Plot the results
outputs = ipinn.eval(XY,activations)
result = getRes(outputs)
exact_st = st_poiss.equation(X.cpu(),Y.cpu())
compare_contours(X.cpu(),Y.cpu(),exact_st,result,colormap = 'cividis',levels=10,nrange=(0,2))

In [ ]:
plot_results3d(X.cpu(),Y.cpu(),exact_st,result)

#### XPINN(VDL)

In [ ]:
#define the model
activations = [F.tanh,F.silu,F.relu]
xpinn = VDLNet(num_domains = 2,dimension = 2,activations = activations,hidden_size = 10,depth = 3)
#Get the loss_function
ls_st.reset()
xpinn.setLoss(lambda x: ls_st.loss(x,'xpinn',cond_func))
#Train the model
xpinn.train(iterations = 10000)

In [ ]:
#Plot the results
outputs = xpinn.eval(XY)
result = getRes(outputs)
compare_contours(X.cpu(),Y.cpu(),exact_st,result,colormap = 'cividis',levels=10,nrange=(0,2))

In [ ]:
plot_results3d(X.cpu(),Y.cpu(),exact_st,result)

### 2. Poisson equation with circular interface

$Ω=[−1,1]×[−1,1]$ with a circular interface, $Γ_{int}=\{x:x^2+y^2=0.25\}$, which subdivides the domain into two regions: <br/>
$Ω_1=\{x:x^2+y^2≤0.25\}⊂Ω$ and $Ω_2=\{x:x^2+y^2>0.25\}⊂Ω$ <br/>
The PDE is:
- $∇⋅(κ_m∇u_m)=0$ in $Ω_m$
- $u_2=1+$ log $2\sqrt{x^2+y^2}$ on $∂Ω^d_2$
- $〚u〛= 0$ on $Γ_{int}$
- $〚κ∇u〛⋅\mathbf{n_2}=−2κ_2$ on $Γ_{int}$

where $∂Ω^d_2=\{x:x=−1∪x=1∪y=−1∪y=1\}$ is the external boundary. $κ_1 = 0.1$ and $κ_2 = 1$
respectively.<br/> These set of Eqs. have the following closed-form solution:
$$ u(x)=\left\{
        \begin{array}{ll}
            1 & \text{in } Ω_1 \\
            1+log2\sqrt{x^2+y^2} & \text{in } Ω_2 \\
        \end{array}
        \right.$$


In [ ]:
#Set the parameters
K = [0.1,1]
#Get loss function
ls_circ = CircLoss(10000,100,100,K)
#Get the exact function
circ = Poisson2DCirc()

In [ ]:
h = 0.001
x = torch.arange(-1, 1+h, h)
y = torch.arange(-1, 1+h, h)
X, Y = torch.meshgrid(x, y)
XY = torch.stack((X, Y), dim=-1).reshape(-1, 2).to(device)
def getResCirc(outputs):
    u1,u2 = outputs[0],outputs[1]
    result = torch.zeros_like(u1).cpu().numpy().reshape(X.shape)
    u1 = u1.cpu().numpy().reshape(X.shape)
    u2 = u2.cpu().numpy().reshape(X.shape)
    mask = X**2 + Y**2 <= 0.25
    result[mask] = u1[mask]
    result[~mask] = u2[~mask]
        
    return result

#### IPINN

In [ ]:
#Get the activation function
def cond_func2(x,condition):
    if(condition == '0'):
        return F.silu(x)
    elif(condition == '1'):
        return F.tanh(x)

In [ ]:
#define the model
ipinn2 = IPINN(dimension = 2,hidden_size = 10,depth = 3)
#Get the loss_function
ipinn2.setLoss(lambda x: ls_circ.loss(x,'ipinn',cond_func2))
#Train the model
ipinn2.train(iterations = 100000)

In [ ]:
activations = [F.silu,F.tanh]
#Plot the results
outputs = ipinn2.eval(XY,activations)
result = getResCirc(outputs)
exact_circ = circ.equation(X.cpu(),Y.cpu())
compare_contours(X.cpu(),Y.cpu(),exact_circ,result,colormap = 'cividis',levels=10,nrange=(0,2))

In [ ]:
plot_results3d(X.cpu(),Y.cpu(),exact_circ,result)

#### XPINN(VDL)

In [ ]:
#define the model
activations = [F.tanh,F.silu,F.relu]
xpinn2 = VDLNet(num_domains = 2,dimension = 2,activations = activations,hidden_size = 10,depth = 3)
#Get the loss_function
ls_circ.reset()
xpinn2.setLoss(lambda x: ls_circ.loss(x,'xpinn',cond_func))
#Train the model
xpinn2.train(iterations = 10000)

In [ ]:
#Plot the results
outputs = xpinn2.eval(XY)
result = getResCirc(outputs)
compare_contours(X.cpu(),Y.cpu(),circ.equation(X.cpu(),Y.cpu()),result,colormap = 'plasma',levels=10,nrange=(0,2))

In [ ]:
plot_results3d(X.cpu(),Y.cpu(),exact_circ,result)

### Results

After running each model for an hour these were the results:

1. Poisson with a straight interface

Metric|IPINN|XPINN(VDL)
---|:---:|:---:
RMSE|$2.0 × 10^{-4}$|$3.1× 10^{-1}$  
Rel. L2|$2.4 \times 10^{-4}$|$3.7× 10^{-1}$

2. Poisson with a circular interface

Metric|IPINN|XPINN(VDL)
---|:---:|:---:
RMSE|$4.2 × 10^{-3}$|$7.4× 10^{-1}$
Rel. L2|$2.9 \times 10^{-3}$|$5.1× 10^{-1}$

All models had hidden layer size of $10$, depth of $3$ and were trained on $10^4$ points

IPINN works better in higher dimensions 